# Download and Transcribe podcasts:


->Download multiple podcasts:------------------------------

In [11]:
import requests
import os
import re
from bs4 import BeautifulSoup
import dateutil.parser

In [13]:
rss_feeds = {'Dave Ramsey': 'https://feeds.megaphone.fm/RM4031649020', 
             'Huberman-Lab': 'https://feeds.megaphone.fm/hubermanlab',
             'Peter Attia MD': 'https://peterattiadrive.libsyn.com/rss',
             'Tim Ferriss Show': 'https://rss.art19.com/tim-ferriss-show',}

In [14]:
for value in rss_feeds.values():
    print(value)

https://feeds.megaphone.fm/RM4031649020
https://feeds.megaphone.fm/hubermanlab
https://peterattiadrive.libsyn.com/rss
https://rss.art19.com/tim-ferriss-show


In [16]:
podcast = 'Tim Ferriss Show'
page = requests.get(rss_feeds[podcast])
soup = BeautifulSoup(page.text, 'xml')

In [17]:
def get_all_podcast_items(rss_feeds):
    all_items = []
    for value in rss_feeds.values():
        page = requests.get(value)
        all_items += BeautifulSoup(page.text, 'xml').find_all('item')
        
    return all_items

In [18]:
all_items = get_all_podcast_items(rss_feeds)

In [19]:
podcast_items = soup.find_all('item')
episodes = [podcast.find('enclosure')['url'] for podcast in podcast_items]
titles = [podcast.find('title').text for podcast in podcast_items]
dates = [podcast.find('pubDate').text for podcast in podcast_items]

In [20]:
def parse_date(date):
    return dateutil.parser.parse(date).strftime('%b-%d-%Y') # Add _%H%M for safety

In [21]:
def get_episodes(podcast_items, search=None, limit=5):
    output = []
    for podcast in podcast_items:
        if len(output) == limit:
            break
        elif search is not None:
            if re.search(search, podcast.find('description').text, re.I):
                output.append(podcast)
        else:
            output.append(podcast)
    
    return output

def get_episode_metadata(podcast_items):
    episode_urls = [podcast.find('enclosure')['url'] for podcast in podcast_items]
    episode_titles = [podcast.find('title').text for podcast in podcast_items]
    episode_release_dates = [parse_date(podcast.find('pubDate').text) for podcast in podcast_items]
    return list(zip(episode_urls, episode_titles, episode_release_dates))

In [22]:
#items = get_episodes(all_items, 'ketamine', limit=5)
metadata = get_episode_metadata(podcast_items)
urls, titles, release_dates = zip(*metadata)

In [23]:
# import datetime

# datetime.datetime.strptime(dates[0], '%a, %d %b %Y %X %f').strftime('%y')

In [25]:
EPISODE_COUNT = 10

In [26]:
path = f'./podcasts/{podcast}'
if not os.path.exists(path):
    os.mkdir(path)

for i in range(EPISODE_COUNT):
    episode_url = episodes[i]
    redirect_url = requests.get(episode_url).url
    results = requests.get(redirect_url)
    file_name = release_dates[i]

    print(file_name)
    with open(f'{path}/{file_name}.mp3', 'wb') as f:
        f.write(results.content)
        
#     print('---')
#     print(titles[i])
#     #file_name = re.sub(r'[/\.\\]', '', titles[i])[:100]
#     file_name = titles[i].replace('/','-').replace('\\','-').replace('.',' ')[:100]

Dec-16-2022
Dec-13-2022
Dec-09-2022
Dec-05-2022
Dec-01-2022
Nov-28-2022
Nov-24-2022
Nov-18-2022
Nov-18-2022
Nov-15-2022


# Transcribe podcasts:

In [27]:
def read_file(filename, chunk_size=5242880):
    with open(filename, 'rb') as _file:
        while True:
            data = _file.read(chunk_size)
            if not data:
                break
            yield data

In [30]:
headers = {'authorization': os.environ['ASSEMBLY_AI_KEY']}

urls = []
files = f'./downloads/{podcast}'
file_names = os.listdir(files)
for file in file_names:
    filename = f'{files}/{file}'
    response = requests.post('https://api.assemblyai.com/v2/upload',
                            headers=headers,
                            data=read_file(filename))
    
    urls.append(response.json())
    

In [31]:
urls

[{'upload_url': 'https://cdn.assemblyai.com/upload/6bd8320b-c7cd-4b47-a105-b314c19c42a3'},
 {'upload_url': 'https://cdn.assemblyai.com/upload/e92b8182-8c50-4dd1-b41a-bdcee4e30137'},
 {'upload_url': 'https://cdn.assemblyai.com/upload/a20a0f15-b9f6-4597-a63f-f65ba5172510'},
 {'upload_url': 'https://cdn.assemblyai.com/upload/3982ce96-8a94-4949-ae9a-671fe548dbd0'},
 {'upload_url': 'https://cdn.assemblyai.com/upload/50ae52fb-1339-4a33-bb09-fbeccf13a412'},
 {'upload_url': 'https://cdn.assemblyai.com/upload/76d6cd87-bf8d-4c11-85ed-df8640c62504'},
 {'upload_url': 'https://cdn.assemblyai.com/upload/6034fa35-86b3-42c9-a58b-291227e9a779'},
 {'upload_url': 'https://cdn.assemblyai.com/upload/c3904fba-35b4-49e9-9ad6-d436e1526bc2'},
 {'upload_url': 'https://cdn.assemblyai.com/upload/4db10690-f48a-428b-8cdd-37d3c10325ad'}]

In [37]:
count=0
output_ids = []
for url in urls:
    print("Transcription #", count)
    endpoint = "https://api.assemblyai.com/v2/transcript"
    json = {
        "audio_url": url['upload_url'],
        "audio_start_from": 300000,
        "audio_end_at": 600000,
    }
    headers = {
        "authorization": os.environ['ASSEMBLY_AI_KEY'],
        "content-type": "application/json"
    }
    response = requests.post(endpoint, json=json, headers=headers)
    print(response)
    output_ids.append(response.json()['id'])
    count+=1

Transcription # 0
<Response [200]>
Transcription # 1
<Response [200]>
Transcription # 2
<Response [200]>
Transcription # 3
<Response [200]>
Transcription # 4
<Response [200]>
Transcription # 5
<Response [200]>
Transcription # 6
<Response [200]>
Transcription # 7
<Response [200]>
Transcription # 8
<Response [200]>


In [42]:
path = './transcripts'
if not os.path.exists(path):
    os.mkdir(path)

for tid in output_ids:
    transcript_url = f'https://api.assemblyai.com/v2/transcript/{tid}'
    response = requests.get(transcript_url, headers=headers)
    print(response.json()['text'])
    raw_text = response.json()['text']
    with open(f'./transcripts/{tid}.txt', 'w') as f:
        f.write(raw_text)

None


TypeError: write() argument must be str, not None

In [44]:
#Transcripted output will be stored in that transcripts directory..